In [ ]:
#Library for reading csv and related operations
import pandas as pd

#for numberical operations
import numpy as np
import random
import math


#Pre processing and evaluation
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error

#LSTM Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed

#Optimizer
from tensorflow.keras.optimizers import Adam
import datetime

In [ ]:
#Settings seeds
np.random.seed(42)
random.seed(12345)
tf.random.set_seed(1234)


We have weekly date from 2017 to 2022 Jan.

 Data shape is 247 rows and 39 columns
3. We would like to forecast n_future=8. Which means 8 weeks ahead in future
4. Train on all data from 0:238 and test on 238:245 (just test on last 8 data rows)

We have columns week, then D1 to Total
(Coherent Hierarchical Time Series Forecasting)

First model: for column D1 TO D31 (Bottom level in the hierarchy )

Second Model: column R1 TO R6 (middle level in the hierarchy)

Third Model: column "Total" (Top level in the hierarchy)

In [ ]:
#Reading the data 
df=pd.read_csv('DRaggDate.csv')
df['week'] = pd.to_datetime(df['week']).dt.date
df

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9,R1,R2,R3,R4,R5,R6,total
0,2017-03-04,0.160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160,0.160
1,2017-03-11,0.275,0.330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.655,NaN,NaN,NaN,0.120,0.240,NaN,0.260,NaN,1.485,NaN,0.530,0.120,NaN,0.275,2.410
2,2017-03-18,1.330,0.130,NaN,NaN,NaN,0.915,NaN,NaN,0.725,NaN,NaN,0.535,NaN,NaN,NaN,0.080,NaN,NaN,NaN,NaN,NaN,NaN,1.725,NaN,NaN,NaN,0.280,1.425,NaN,NaN,NaN,4.005,NaN,0.535,0.360,0.915,1.330,7.145
3,2017-03-25,1.245,0.210,NaN,0.960,NaN,3.185,0.320,NaN,NaN,NaN,NaN,0.415,NaN,NaN,NaN,0.020,NaN,NaN,NaN,NaN,NaN,NaN,1.575,NaN,NaN,NaN,1.100,0.435,NaN,NaN,NaN,2.220,NaN,0.735,1.120,4.145,1.245,9.465
4,2017-04-01,0.840,0.090,NaN,0.640,NaN,2.645,NaN,NaN,NaN,NaN,NaN,0.535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.480,NaN,NaN,NaN,1.230,0.050,NaN,NaN,NaN,0.620,NaN,0.535,1.230,3.285,0.840,6.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-12-18,47.210,11.715,3.260,33.585,23.605,33.195,122.690,30.750,11.325,5.125,29.485,23.080,5.235,23.085,31.225,43.960,29.145,47.105,4.220,2.770,20.915,18.390,27.850,3.36,4.540,8.560,116.430,54.240,105.635,33.870,183.205,165.375,116.660,504.800,233.720,71.000,47.210,1141.500
243,2021-12-25,35.845,10.025,2.170,28.210,14.550,23.620,84.720,20.715,8.825,3.980,23.680,16.480,3.545,11.340,16.695,29.280,20.060,32.410,3.735,1.355,15.510,10.675,19.715,2.72,3.255,6.600,78.875,35.505,71.445,23.595,121.000,113.570,76.485,337.395,161.275,55.565,35.845,783.430
244,2022-01-15,40.470,8.810,1.895,27.450,20.455,20.465,98.270,27.355,9.200,4.295,26.240,18.325,4.255,15.715,25.970,38.325,19.555,37.965,3.555,2.215,16.990,17.745,18.280,1.11,3.330,7.435,90.055,45.180,75.600,25.765,134.370,129.905,91.870,380.245,192.685,51.470,40.470,889.465
245,2022-01-22,45.205,11.350,2.230,31.760,24.495,32.235,111.490,30.960,10.640,3.845,28.755,18.380,5.030,19.555,29.855,43.740,22.270,46.785,4.225,3.140,22.085,18.985,25.210,1.44,3.795,9.460,107.420,52.565,96.100,33.565,153.320,160.965,105.090,445.630,224.780,68.220,45.205,1052.755


In [ ]:
#Function to calculate the inverse different of 2 values
def diff_inverse(obj_prev, value):
    return value + obj_prev

#To Calculate Mean absolute error
def MAE(x,y):
    result=0
    #looping from 0 to length(x)
    for i in range(len(x)):
    #taking the absolute of difference
        result += abs((x[i]-y[i])/x[i])
    #dividing by length
    result /= len(x)
    #multiplying it by 100
    result *= 100
    return result

#To Calculate Mean Squared Error
def RMSE(x,y):
    result=0
    for i in range(len(x)):
        result += ((x[i]-y[i])/x[i])**2
    #dividing by length
    result /= len(x)
    return result

#To Calculate Root Mean Squared Error
def dRMSE(y_true, y_pred):
    #difference wrt rows (0 axis)
    dy_true=np.diff(y_true, axis=0)
    dy_pred=np.diff(y_pred, axis=0)
    result=0
    for i in range(len(dy_true)):
        result += ((dy_true[i]-dy_pred[i])/dy_true[i])**2
    result /= len(dy_true)
    # Taking square root
    result = np.sqrt(result)
    return result

In [ ]:
#Converting the series to samples for the LSTM supervised learning method
#Function to split the series into past and future observations
#Here:
# n_past ==> no of past observations
# n_future ==> no of future observations 
def split_series(series, n_past, n_future):
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        #Adding past and future values to the list
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [ ]:
#Filling the missing values
df = df.bfill(axis ='rows')

**Level - 3 (Bottom)**


Total visitors according to city visit, with a total of 56 visits

In [ ]:
#Choosing first 31 columns
first_daily_df = df.iloc[:,1:32]
first_daily_df[:-9] # chunk that will be used for training

,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
0,0.160,0.330,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.530,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,0.655,28.345,0.715,0.050,0.120,0.240,2.230,0.260,0.060
1,0.275,0.330,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.530,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,0.655,28.345,0.715,0.050,0.120,0.240,2.230,0.260,0.060
2,1.330,0.130,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.535,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,1.725,28.345,0.715,0.050,0.280,1.425,2.230,0.125,0.060
3,1.245,0.210,0.240,0.960,10.570,3.185,0.320,0.190,0.040,0.460,13.360,0.415,1.270,0.130,0.020,0.020,0.110,16.510,2.500,0.260,0.11,0.020,1.575,28.345,0.715,0.050,1.100,0.435,2.230,0.125,0.060
4,0.840,0.090,0.240,0.640,10.570,2.645,0.160,0.190,0.040,0.460,13.360,0.535,1.270,0.130,0.020,0.720,0.110,16.510,2.500,0.260,0.11,0.020,0.480,28.345,0.715,0.050,1.230,0.050,2.230,0.125,0.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,48.255,9.645,2.855,33.355,22.155,34.780,108.530,33.765,9.310,6.010,28.095,23.370,4.205,20.570,29.100,40.065,26.335,41.965,5.775,2.060,15.80,18.545,25.530,1.000,3.870,8.900,118.415,53.955,96.550,31.415,186.190
234,51.340,11.570,3.075,34.630,23.945,34.045,111.265,34.360,9.800,1.750,27.010,23.165,4.040,17.555,29.485,38.680,27.265,43.045,5.385,2.815,16.36,16.355,26.835,1.785,4.670,9.380,117.350,55.160,95.920,34.055,201.070
235,48.685,10.325,3.295,32.765,22.315,32.995,112.940,33.365,7.475,1.820,28.370,21.460,3.750,17.960,28.775,35.800,25.970,44.260,5.190,3.245,16.72,17.315,27.055,1.430,3.955,8.740,111.570,54.010,92.425,28.110,183.220
236,47.365,12.695,3.000,35.100,22.820,34.570,116.585,32.800,7.630,1.975,30.755,22.550,4.970,18.645,32.540,39.775,29.405,43.360,5.050,3.400,19.98,17.645,27.315,1.000,4.370,9.300,118.850,57.270,102.140,29.580,189.345


In [ ]:
# Checking the shape of the data
first_daily_df.shape

(247, 31)

In [ ]:
# First discrete difference of element. 
# Calculates the difference of a Dataframe element compared with another element in the Dataframe (default is element in previous row).
difference = first_daily_df.diff()
difference[:-9]

,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.115,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,1.055,-0.200,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.070,0.000,0.000,0.000,0.160,1.185,0.000,-0.135,0.000
3,-0.085,0.080,0.000,0.000,0.000,2.270,0.000,0.000,-0.685,0.000,0.000,-0.120,0.000,0.000,0.000,-0.060,0.000,0.000,0.000,0.000,0.000,0.000,-0.150,0.000,0.000,0.000,0.820,-0.990,0.000,0.000,0.000
4,-0.405,-0.120,0.000,-0.320,0.000,-0.540,-0.160,0.000,0.000,0.000,0.000,0.120,0.000,0.000,0.000,0.700,0.000,0.000,0.000,0.000,0.000,0.000,-1.095,0.000,0.000,0.000,0.130,-0.385,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,-0.215,0.430,0.605,-0.335,-1.965,3.205,2.685,2.430,-4.295,-0.005,0.890,-0.045,0.685,-1.155,1.415,0.245,0.720,0.460,-0.125,0.015,-0.535,-3.485,-0.460,-0.300,-1.110,0.685,5.295,1.160,-0.370,0.485,9.400
234,3.085,1.925,0.220,1.275,1.790,-0.735,2.735,0.595,0.490,-4.260,-1.085,-0.205,-0.165,-3.015,0.385,-1.385,0.930,1.080,-0.390,0.755,0.560,-2.190,1.305,0.785,0.800,0.480,-1.065,1.205,-0.630,2.640,14.880
235,-2.655,-1.245,0.220,-1.865,-1.630,-1.050,1.675,-0.995,-2.325,0.070,1.360,-1.705,-0.290,0.405,-0.710,-2.880,-1.295,1.215,-0.195,0.430,0.360,0.960,0.220,-0.355,-0.715,-0.640,-5.780,-1.150,-3.495,-5.945,-17.850
236,-1.320,2.370,-0.295,2.335,0.505,1.575,3.645,-0.565,0.155,0.155,2.385,1.090,1.220,0.685,3.765,3.975,3.435,-0.900,-0.140,0.155,3.260,0.330,0.260,-0.430,0.415,0.560,7.280,3.260,9.715,1.470,6.125


In [ ]:
#Features
n_past = 1
n_future = 52 # Predicting the next
no_of_features = 31

#Splitting the data into testing and training such that

# remaining 9 are set as testing data which we will compare with our forecasting predictions
train_dataF,test_dataF = difference[0:-9], difference[-9:]  
train_dataF.shape,test_dataF.shape

((238, 31), (9, 31))

In [ ]:
train = train_dataF
scalers={}

#Scaling the values for faster training of the models
for i in train_dataF.columns:
    #Using MinMaxScaler (it converts data in range 0-1)
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

test = test_dataF
for i in train_dataF.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s


#Storing the training split series into the variables
X_train, y_train = split_series(train.values,n_past, 2)
print(X_train.shape)
print(y_train.shape)
#Reshaping according to LSTM model input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],no_of_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], no_of_features))

#Finding the None (Nan) Values
where_are_NaNs = np.isnan(X_train)

#Setting None to 0
X_train[where_are_NaNs] = 0

#Splitting the test values as well wrt past and future
X_test, y_test = split_series(test.values,n_past, 2)

X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],no_of_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], no_of_features))

(236, 1, 31)
(236, 2, 31)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [ ]:
#raw data 
newdataX,newdatay=split_series(first_daily_df.values,n_past,2)

In [ ]:
newdatay = newdatay[-9:, :]

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

# we use this as actual value (Testing set)
newdatay.shape

(236, 1, 31)
(236, 2, 31)
(7, 1, 31)
(7, 2, 31)


(9, 2, 31)

In [ ]:
#LSTM Model Architecture
model = Sequential()
model.add(LSTM(10, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
model.add(LSTM(7))
model.add(RepeatVector(2))
model.add(LSTM(7, return_sequences=True))
model.add(LSTM(10, return_sequences=True))
model.add(TimeDistributed(Dense(no_of_features)))

#comiling the model with adam optimer and loss mean squared error
model.compile(optimizer='adam', loss='mse')

#printing summary of model
model.summary()

#Fitting the model
fit_history1=model.fit(X_train,y_train,epochs=300,validation_split=0.33,batch_size=30,verbose=2,shuffle=False)

#Saving the model
model.save("model(8w1).h5") # change here with number of step

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 10)             1680      
                                                                 
 lstm_1 (LSTM)               (None, 7)                 504       
                                                                 
 repeat_vector (RepeatVector  (None, 2, 7)             0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 2, 7)              420       
                                                                 
 lstm_3 (LSTM)               (None, 2, 10)             720       
                                                                 
 time_distributed (TimeDistr  (None, 2, 31)            341       
 ibuted)                                                

In [ ]:
#Predictions on testing set - Next forecast
model_pred=model.predict(X_test)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [ ]:
#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

In [ ]:
#Plots for losses 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#History for model 1 (first 56 columns)
history = fit_history1
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=loss, name="Training Loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=val_loss, name="Validation Loss"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Training and Validation Loss"
)

# Set x-axis title
fig.update_xaxes(title_text="Epochs")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Training Loss</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Validation Loss</b>", secondary_y=True)

fig.show()

In [ ]:
#Checking the metrics - RMSE and MAE

print('MAE_0',(MAE(newdatay[0,:], model_pred[0,:])).mean()) # for week 238
print('RMSE_0', (RMSE(newdatay[0,:], model_pred[0,:])).mean())
print('dRMSE_0', (RMSE(newdatay[0,:], model_pred[0,:])).mean())


print('MAE_1', (MAE(newdatay[1,:], model_pred[1,:])).mean()) # for  week 239

print('MAE_2', (MAE(newdatay[2,:], model_pred[2,:])).mean()) #  for  week 240

print('MAE_3', (MAE(newdatay[3,:], model_pred[3,:])).mean()) # for  week 241

print('MAE_4', (MAE(newdatay[4,:], model_pred[4,:])).mean())

print('MAE_5', (MAE(newdatay[5,:], model_pred[5,:])).mean())

print()
print('Full MAE', (MAE(newdatay[:-2], model_pred)).mean())
print('Full RMSE', (RMSE(newdatay[:-2], model_pred)).mean())
#print('dRMSE', (RMSE(newdatay, model_pred)).mean())

MAE_0 0.6527964128993387
RMSE_0 0.000274130063425723
dRMSE_0 0.000274130063425723
MAE_1 0.5397397184283824
MAE_2 0.6617871189145474
MAE_3 0.6530840348444901
MAE_4 0.46428136505909295
MAE_5 29.77926448638553

Full MAE 4.786010633939956
Full RMSE 0.12454503554309168


In [ ]:
# Forecasting Visualizations

In [ ]:
#Lets see the first 238 weeks data - years which we've trained our model for.
first_daily_df[:-9]

,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
0,0.160,0.330,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.530,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,0.655,28.345,0.715,0.050,0.120,0.240,2.230,0.260,0.060
1,0.275,0.330,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.530,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,0.655,28.345,0.715,0.050,0.120,0.240,2.230,0.260,0.060
2,1.330,0.130,0.240,0.960,10.570,0.915,0.320,0.190,0.725,0.460,13.360,0.535,1.270,0.130,0.020,0.080,0.110,16.510,2.500,0.260,0.11,0.020,1.725,28.345,0.715,0.050,0.280,1.425,2.230,0.125,0.060
3,1.245,0.210,0.240,0.960,10.570,3.185,0.320,0.190,0.040,0.460,13.360,0.415,1.270,0.130,0.020,0.020,0.110,16.510,2.500,0.260,0.11,0.020,1.575,28.345,0.715,0.050,1.100,0.435,2.230,0.125,0.060
4,0.840,0.090,0.240,0.640,10.570,2.645,0.160,0.190,0.040,0.460,13.360,0.535,1.270,0.130,0.020,0.720,0.110,16.510,2.500,0.260,0.11,0.020,0.480,28.345,0.715,0.050,1.230,0.050,2.230,0.125,0.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,48.255,9.645,2.855,33.355,22.155,34.780,108.530,33.765,9.310,6.010,28.095,23.370,4.205,20.570,29.100,40.065,26.335,41.965,5.775,2.060,15.80,18.545,25.530,1.000,3.870,8.900,118.415,53.955,96.550,31.415,186.190
234,51.340,11.570,3.075,34.630,23.945,34.045,111.265,34.360,9.800,1.750,27.010,23.165,4.040,17.555,29.485,38.680,27.265,43.045,5.385,2.815,16.36,16.355,26.835,1.785,4.670,9.380,117.350,55.160,95.920,34.055,201.070
235,48.685,10.325,3.295,32.765,22.315,32.995,112.940,33.365,7.475,1.820,28.370,21.460,3.750,17.960,28.775,35.800,25.970,44.260,5.190,3.245,16.72,17.315,27.055,1.430,3.955,8.740,111.570,54.010,92.425,28.110,183.220
236,47.365,12.695,3.000,35.100,22.820,34.570,116.585,32.800,7.630,1.975,30.755,22.550,4.970,18.645,32.540,39.775,29.405,43.360,5.050,3.400,19.98,17.645,27.315,1.000,4.370,9.300,118.850,57.270,102.140,29.580,189.345


In [ ]:
#Actual values for last 8 weeks - testing set
actual_df = pd.DataFrame(newdatay[:,1,:], index = list(range(238,247)))
for i in range(len(test.columns)):
    actual_df = actual_df.rename(columns={i: test.columns[i]}) 

actual_df["week"] = df.iloc[238:,0]
actual_df["week"] = pd.to_datetime(actual_df['week']).dt.date
cols = actual_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
actual_df= actual_df[cols]
actual_df

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
238,2021-11-20,50.540,9.490,3.200,30.770,25.305,36.540,123.000,33.485,10.740,3.275,29.465,22.330,6.100,22.050,32.715,42.765,27.190,46.905,4.175,2.730,20.365,9.430,29.040,1.800,4.015,9.325,115.355,54.385,104.850,30.865,206.400
239,2021-11-27,50.080,11.005,3.150,31.180,24.290,35.035,120.370,31.545,12.630,4.055,28.490,22.835,4.880,23.120,33.595,40.005,26.900,45.740,5.160,1.820,20.265,19.315,28.535,1.445,4.000,8.230,114.235,53.725,110.345,32.225,205.155
240,2021-12-04,46.300,10.410,3.015,31.580,25.480,32.985,121.320,34.030,12.390,3.960,29.365,20.455,4.930,21.735,28.820,41.530,28.390,45.990,4.950,2.315,21.660,16.450,29.285,1.110,4.345,8.265,117.385,57.630,106.645,31.895,165.695
241,2021-12-11,45.505,11.925,2.885,33.240,24.905,33.400,122.005,33.600,12.790,3.710,29.645,20.675,5.325,22.715,32.740,43.895,31.375,47.365,4.775,2.445,21.575,18.780,29.150,1.275,4.360,8.325,120.640,55.880,106.810,34.990,181.315
242,2021-12-18,47.210,11.715,3.260,33.585,23.605,33.195,122.690,30.750,11.325,5.125,29.485,23.080,5.235,23.085,31.225,43.960,29.145,47.105,4.220,2.770,20.915,18.390,27.850,3.360,4.540,8.560,116.430,54.240,105.635,33.870,183.205
243,2021-12-25,35.845,10.025,2.170,28.210,14.550,23.620,84.720,20.715,8.825,3.980,23.680,16.480,3.545,11.340,16.695,29.280,20.060,32.410,3.735,1.355,15.510,10.675,19.715,2.720,3.255,6.600,78.875,35.505,71.445,23.595,121.000
244,2022-01-15,40.470,8.810,1.895,27.450,20.455,20.465,98.270,27.355,9.200,4.295,26.240,18.325,4.255,15.715,25.970,38.325,19.555,37.965,3.555,2.215,16.990,17.745,18.280,1.110,3.330,7.435,90.055,45.180,75.600,25.765,134.370
245,2022-01-22,45.205,11.350,2.230,31.760,24.495,32.235,111.490,30.960,10.640,3.845,28.755,18.380,5.030,19.555,29.855,43.740,22.270,46.785,4.225,3.140,22.085,18.985,25.210,1.440,3.795,9.460,107.420,52.565,96.100,33.565,153.320
246,2022-01-29,45.750,10.405,2.460,32.490,24.325,32.645,117.965,31.165,11.725,2.755,30.750,21.355,5.430,22.600,31.440,47.065,25.535,48.650,3.935,3.310,23.160,17.580,28.200,2.180,3.910,9.110,113.200,53.020,104.755,34.495,173.575


In [ ]:
#Predicted values for last 8 weeks
forecast1_df = pd.DataFrame(model_pred[1:,0,:], index = list(range(238,244))) 
forecast2_df = pd.DataFrame(model_pred[-5:-2,1,:], index = list(range(244,247))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 

merged["week"] = df.iloc[238:,0]
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
238,2021-11-20,50.668198,9.545790,3.203572,30.806177,25.350321,36.660938,123.193691,33.511412,11.019328,3.164666,29.505369,22.406938,6.105972,22.069822,32.807314,42.922753,27.427745,46.965789,4.188084,2.752429,20.439900,9.592544,29.074592,2.084618,4.005563,9.337839,115.516596,54.505692,105.119394,31.009296,206.617377
239,2021-11-27,50.208198,11.060790,3.153572,31.216177,24.335321,35.155938,120.563691,31.571412,12.909328,3.944666,28.530369,22.911938,4.885972,23.139822,33.687314,40.162753,27.137745,45.800789,5.173084,1.842429,20.339900,19.477544,28.569592,1.729618,3.990563,8.242839,114.396596,53.845692,110.614394,32.369296,205.372377
240,2021-12-04,46.428198,10.465790,3.018572,31.616177,25.525321,33.105938,121.513691,34.056412,12.669328,3.849666,29.405369,20.531938,4.935972,21.754822,28.912314,41.687753,28.627745,46.050789,4.963084,2.337429,21.734900,16.612544,29.319592,1.394618,4.335563,8.277839,117.546596,57.750692,106.914394,32.039296,165.912377
241,2021-12-11,45.633198,11.980790,2.888572,33.276177,24.950321,33.520938,122.198691,33.626412,13.069328,3.599666,29.685369,20.751938,5.330972,22.734822,32.832314,44.052753,31.612745,47.425789,4.788084,2.467429,21.649900,18.942544,29.184592,1.559618,4.350563,8.337839,120.801596,56.000692,107.079394,35.134296,181.532377
242,2021-12-18,47.338198,11.770790,3.263572,33.621177,23.650321,33.315938,122.883691,30.776412,11.604328,5.014666,29.525369,23.156938,5.240972,23.104822,31.317314,44.117753,29.382745,47.165789,4.233084,2.792429,20.989900,18.552544,27.884592,3.644618,4.530563,8.572839,116.591596,54.360692,105.904394,34.014296,183.422377
243,2021-12-25,35.973198,10.080790,2.173572,28.246176,14.595321,23.740938,84.913689,20.741412,9.104327,3.869666,23.720368,16.556938,3.550972,11.359822,16.787315,29.437752,20.297745,32.470789,3.748084,1.377430,15.584900,10.837543,19.749592,3.004618,3.245563,6.612839,79.036596,35.625691,71.714394,23.739296,121.217381
244,2022-01-15,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
245,2022-01-22,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
246,2022-01-29,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154


# Forecasting for next 52 weeks in future

In [ ]:
ffs1 = actual_df.iloc[-1:,0]+datetime.timedelta(days=7)
ffs2 = ffs1+datetime.timedelta(days=7)
ffs3 = ffs2+datetime.timedelta(days=7)
ffs4 = ffs3+datetime.timedelta(days=7)
ffs5 = ffs4+datetime.timedelta(days=7)
ffs6 = ffs5+datetime.timedelta(days=7)
ffs7 = ffs6+datetime.timedelta(days=7)
ffs8 = ffs7+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs1, ffs2, ffs3, ffs4, ffs5, ffs6, ffs7, ffs8], axis=0)
future_weeks.index = range(247,255)
future_weeks

247    2022-02-05
248    2022-02-12
249    2022-02-19
250    2022-02-26
251    2022-03-05
252    2022-03-12
253    2022-03-19
254    2022-03-26
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(247,248))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(248,255))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
247,2022-02-05,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
248,2022-02-12,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
249,2022-02-19,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
250,2022-02-26,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
251,2022-03-05,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
252,2022-03-12,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
253,2022-03-19,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
254,2022-03-26,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
X_ff, y_ff = split_series(merged.iloc[:,1:].values,n_past, 1)

#Predictions on testing set - Next forecast
model_pred=model.predict(X_ff)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

In [ ]:
ffs9 = ffs8+datetime.timedelta(days=7)
ffs10 = ffs9+datetime.timedelta(days=7)
ffs11 = ffs10+datetime.timedelta(days=7)
ffs12 = ffs11+datetime.timedelta(days=7)
ffs13 = ffs12+datetime.timedelta(days=7)
ffs14 = ffs13+datetime.timedelta(days=7)
ffs15 = ffs14+datetime.timedelta(days=7)
ffs16 = ffs15+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs9, ffs10, ffs11, ffs12, ffs13, ffs14, ffs15, ffs16], axis=0)
future_weeks.index = range(255,263)
future_weeks

255    2022-04-02
256    2022-04-09
257    2022-04-16
258    2022-04-23
259    2022-04-30
260    2022-05-07
261    2022-05-14
262    2022-05-21
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - more 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(255,256))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(256,263))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
255,2022-04-02,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
256,2022-04-09,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
257,2022-04-16,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
258,2022-04-23,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
259,2022-04-30,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
260,2022-05-07,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
261,2022-05-14,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
262,2022-05-21,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
X_ff, y_ff = split_series(merged.iloc[:,1:].values,n_past, 1)

In [ ]:
#Predictions on testing set - Next forecast
model_pred=model.predict(X_ff)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

In [ ]:
ffs17 = ffs16+datetime.timedelta(days=7)
ffs18 = ffs17+datetime.timedelta(days=7)
ffs19 = ffs18+datetime.timedelta(days=7)
ffs20 = ffs19+datetime.timedelta(days=7)
ffs21 = ffs20+datetime.timedelta(days=7)
ffs22 = ffs21+datetime.timedelta(days=7)
ffs23 = ffs22+datetime.timedelta(days=7)
ffs24 = ffs23+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs17, ffs18, ffs19, ffs20, ffs21, ffs22, ffs23, ffs24], axis=0)
future_weeks.index = range(263,271)
future_weeks

263    2022-05-28
264    2022-06-04
265    2022-06-11
266    2022-06-18
267    2022-06-25
268    2022-07-02
269    2022-07-09
270    2022-07-16
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - more 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(263,264))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(264,271))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
263,2022-05-28,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
264,2022-06-04,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
265,2022-06-11,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
266,2022-06-18,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
267,2022-06-25,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
268,2022-07-02,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
269,2022-07-09,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
270,2022-07-16,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
X_ff, y_ff = split_series(merged.iloc[:,1:].values,n_past, 1)

#Predictions on testing set - Next forecast
model_pred=model.predict(X_ff)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

ffs25 = ffs24+datetime.timedelta(days=7)
ffs26 = ffs25+datetime.timedelta(days=7)
ffs27 = ffs26+datetime.timedelta(days=7)
ffs28 = ffs27+datetime.timedelta(days=7)
ffs29 = ffs28+datetime.timedelta(days=7)
ffs30 = ffs29+datetime.timedelta(days=7)
ffs31 = ffs30+datetime.timedelta(days=7)
ffs32 = ffs31+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs25, ffs26, ffs27, ffs28, ffs29, ffs30, ffs31, ffs32], axis=0)
future_weeks.index = range(271,279)
future_weeks

271    2022-07-23
272    2022-07-30
273    2022-08-06
274    2022-08-13
275    2022-08-20
276    2022-08-27
277    2022-09-03
278    2022-09-10
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - more 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(271,272))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(272,279))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
271,2022-07-23,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
272,2022-07-30,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
273,2022-08-06,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
274,2022-08-13,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
275,2022-08-20,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
276,2022-08-27,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
277,2022-09-03,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
278,2022-09-10,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
X_ff, y_ff = split_series(merged.iloc[:,1:].values,n_past, 1)

#Predictions on testing set - Next forecast
model_pred=model.predict(X_ff)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

ffs33 = ffs32+datetime.timedelta(days=7)
ffs34 = ffs33+datetime.timedelta(days=7)
ffs35 = ffs34+datetime.timedelta(days=7)
ffs36 = ffs35+datetime.timedelta(days=7)
ffs37 = ffs36+datetime.timedelta(days=7)
ffs38 = ffs37+datetime.timedelta(days=7)
ffs39 = ffs38+datetime.timedelta(days=7)
ffs40 = ffs39+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs33, ffs34, ffs35, ffs36, ffs37, ffs38, ffs39, ffs40], axis=0)
future_weeks.index = range(279,287)
future_weeks

279    2022-09-17
280    2022-09-24
281    2022-10-01
282    2022-10-08
283    2022-10-15
284    2022-10-22
285    2022-10-29
286    2022-11-05
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - more 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(279,280))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(280,287))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
279,2022-09-17,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
280,2022-09-24,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
281,2022-10-01,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
282,2022-10-08,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
283,2022-10-15,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
284,2022-10-22,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
285,2022-10-29,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
286,2022-11-05,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
X_ff, y_ff = split_series(merged.iloc[:,1:].values,n_past, 1)

#Predictions on testing set - Next forecast
model_pred=model.predict(X_ff)

#Inverse Scaling of the predicted values to get the real value from 0-1 range
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

#Taking inverse of difference as well
model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)

ffs41 = ffs40+datetime.timedelta(days=7)
ffs42 = ffs41+datetime.timedelta(days=7)
ffs43 = ffs42+datetime.timedelta(days=7)
ffs44 = ffs43+datetime.timedelta(days=7)
ffs45 = ffs44+datetime.timedelta(days=7)
ffs46 = ffs45+datetime.timedelta(days=7)
ffs47 = ffs46+datetime.timedelta(days=7)
ffs48 = ffs47+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs41, ffs42, ffs43, ffs44, ffs45, ffs46, ffs47, ffs48], axis=0)
future_weeks.index = range(287,295)
future_weeks

287    2022-11-12
288    2022-11-19
289    2022-11-26
290    2022-12-03
291    2022-12-10
292    2022-12-17
293    2022-12-24
294    2022-12-31
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - more 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(287,288))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(288,295))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,D1,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D2,D20,D21,D22,D23,D24,D25,D26,D27,D28,D29,D3,D30,D31,D4,D5,D6,D7,D8,D9
287,2022-11-12,48.498198,10.075790,2.883572,31.531177,24.750321,33.230938,116.958691,34.031412,9.129328,2.629666,30.405369,23.036938,5.020972,19.784822,32.362314,44.567753,28.127745,42.115789,5.423084,3.412429,18.374900,9.927544,26.084592,1.764618,4.110563,8.477839,117.626596,55.155692,103.119394,28.499296,200.737377
288,2022-11-19,50.655737,9.538337,3.198745,30.758866,25.303354,36.588553,123.058389,33.483137,11.014318,3.082418,29.491258,22.342602,6.099335,22.031807,32.728440,42.881939,27.371485,46.902531,4.177917,2.732563,20.411069,9.560373,29.079395,1.684049,3.998405,9.322465,115.479220,54.410904,104.948386,30.946863,206.579154
289,2022-11-26,50.195737,11.053337,3.148745,31.168866,24.288354,35.083553,120.428389,31.543137,12.904318,3.862418,28.516258,22.847602,4.879335,23.101807,33.608440,40.121939,27.081485,45.737531,5.162917,1.822563,20.311069,19.445373,28.574395,1.329049,3.983405,8.227465,114.359220,53.750904,110.443386,32.306863,205.334154
290,2022-12-03,46.415737,10.458337,3.013745,31.568866,25.478354,33.033553,121.378389,34.028137,12.664318,3.767418,29.391258,20.467602,4.929335,21.716807,28.833440,41.646939,28.571485,45.987531,4.952917,2.317563,21.706069,16.580373,29.324395,0.994049,4.328405,8.262465,117.509220,57.655904,106.743386,31.976863,165.874154
291,2022-12-10,45.620737,11.973337,2.883745,33.228866,24.903354,33.448553,122.063389,33.598137,13.064318,3.517418,29.671258,20.687602,5.324335,22.696807,32.753440,44.011939,31.556485,47.362531,4.777917,2.447563,21.621069,18.910373,29.189395,1.159049,4.343405,8.322465,120.764220,55.905904,106.908386,35.071863,181.494154
292,2022-12-17,47.325737,11.763337,3.258745,33.573866,23.603354,33.243553,122.748389,30.748137,11.599318,4.932418,29.511258,23.092602,5.234335,23.066807,31.238440,44.076939,29.326485,47.102531,4.222917,2.772563,20.961069,18.520373,27.889395,3.244049,4.523405,8.557465,116.554220,54.265904,105.733386,33.951863,183.384154
293,2022-12-24,35.960737,10.073337,2.168745,28.198866,14.548354,23.668553,84.778389,20.713137,9.099318,3.787418,23.706258,16.492602,3.544335,11.321807,16.708440,29.396939,20.241485,32.407531,3.737917,1.357563,15.556069,10.805373,19.754395,2.604049,3.238405,6.597465,78.999220,35.530904,71.543386,23.676863,121.179154
294,2022-12-31,40.585737,8.858337,1.893745,27.438865,20.453354,20.513554,98.328389,27.353137,9.474318,4.102420,26.266258,18.337602,4.254335,15.696807,25.983441,38.441939,19.736485,37.962531,3.557917,2.217563,17.036070,17.875373,18.319395,0.994049,3.313405,7.432465,90.179220,45.205904,75.698386,25.846864,134.549154


In [ ]:
#Visualizations

In [ ]:
# Setting x-axis for visualization of plots
x = list(range(0, 31))
x1 = list(range(max(x)+1, max(x)+32))
x2 = list(range(max(x1)+1, max(x1)+32))
x3 = list(range(max(x2)+1, max(x2)+32))
x4 = list(range(max(x3)+1, max(x3)+32))
x5 = list(range(max(x4)+1, max(x4)+32))
x6 = list(range(max(x5)+1, max(x5)+32))
x7 = list(range(max(x6)+1, max(x6)+32))
x8 = list(range(max(x7)+1, max(x7)+32))
x9 = list(range(max(x8)+1, max(x8)+32))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

prev_val1 = first_daily_df.values[-2] # Week No 245
prev_val2 = first_daily_df.values[-1] # Week No 246

forc_val1 = model_pred[:,0,:][0] # Predicted # Week No 247
forc_val2 = model_pred[:,1,:][1] # Predicted # Week No 248
forc_val3 = model_pred[:,1,:][2] # Predicted # Week No 249
forc_val4 = model_pred[:,1,:][3] # Predicted # Week No 250

# Add traces
fig.add_trace(
    go.Scatter(x = x, y=prev_val1, name="Test - Week 245",line=dict(color="#0000ff")),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x= x1, y=prev_val2, name="Test - Week 246",line=dict(color="#0000ff")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x2, y=forc_val1, name="Forecast - Week 247",line=dict(color="darkgreen")),
    secondary_y=True,
)


fig.add_trace(
    go.Scatter(x= x3, y=forc_val2, name="Forecast - Week 248",line=dict(color="darkgreen")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x4, y=forc_val3, name="Forecast - Week 249",line=dict(color="darkgreen")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x5, y=forc_val4, name="Forecast - Week 250",line=dict(color="darkgreen")),
    secondary_y=True,
)
#can be forecasted upto 36 quarters!

# Add figure title
fig.update_layout(
    title_text="Visualization of previous 2 weeks vs next 4 weeks - forecasting"
)

# Set x-axis title
fig.update_xaxes(title_text="examples")

# Set y-axes titles
fig.update_yaxes(title_text="<b>actual features</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>predictions features</b>", secondary_y=True)

fig.show()

**Level 2 (middle):** 

Second Model: column R1 TO R6 (middle level in the hierarchy)

# Taking the next columns from R1-R6

In [ ]:
first_daily_df=df.iloc[:,32:38]
difference=first_daily_df.diff()
difference[:-9]

,R1,R2,R3,R4,R5,R6
0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000,0.000,0.000,0.000,0.000,0.115
2,2.520,0.000,0.005,0.240,0.000,1.055
3,-1.785,0.000,0.200,0.760,3.230,-0.085
4,-1.600,0.000,-0.200,0.110,-0.860,-0.405
...,...,...,...,...,...,...
233,-4.630,6.235,10.945,1.300,2.745,-0.215
234,10.110,-1.030,14.655,-4.365,0.150,3.085
235,-11.645,-3.995,-19.580,-6.695,-3.110,-2.655
236,8.175,7.845,19.385,17.645,3.770,-1.320


In [ ]:
#Features
n_past = 1
n_future = 8
no_of_features = 6 # now we have taken 6 features

train_dataF,test_dataF = difference[0:-9], difference[-9:]  

#Scaling the values for faster training of the models.
train = train_dataF
scalers={}

for i in train_dataF.columns:
    #MixMaxScler converts into [0 1]
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s

test = test_dataF
for i in train_dataF.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s



X_train, y_train = split_series(train.values,n_past, 2)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],no_of_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], no_of_features))
where_are_NaNs = np.isnan(X_train)
X_train[where_are_NaNs] = 0

X_test, y_test = split_series(test.values,n_past, 2)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],no_of_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], no_of_features))
#print(y_train,X_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [ ]:
#Shape of our training and testing data without zero padding
print(X_train.shape)
print(X_test.shape)
#Here we see that we now have 28 columns in our training/testing input data.

(236, 1, 6)
(7, 1, 6)


In [ ]:
#Padding zeros to overcome the dimension errors

#Beacuse we need to need have 31 columns to re-use the [model no 1] 

# So we pad 25 columns of zeros (z_train) to our data using concatenate function
z_train=np.zeros((238-2,n_past,25))
X_train = np.concatenate([X_train, z_train], -1)
print(X_train.shape)
print(y_train.shape)
z_test=np.zeros((9-2,n_past,25))
X_test = np.concatenate([X_test, z_test], -1)
print(X_test.shape)
print(y_test.shape)

#Splitting the data for testing wrt past and future
newdataX,newdatay=split_series(first_daily_df.values,n_past, 2)
newdatay = newdatay[-9:, :]
newdataX.shape

(236, 1, 31)
(236, 2, 6)
(7, 1, 31)
(7, 2, 6)


(245, 1, 6)

In [ ]:
#Loading the previous saved model as we are performing the Hierarchical Time Series Forecasting
from keras.models import Model
from keras.models import load_model

#This was trained with first 31 columns, we load it and re-train it
model2=load_model('model(8w1).h5')

# we set the 5 layers of model trainable false to re-train them with next columns for Hierarchical forecasting

#trainable to False moves all the layer's weights from trainable to non-trainable. 
#This is called "freezing" the layer: the state of a frozen layer won't be updated 
#during training
#So it means we will use those pre-trained weights (transfer learning)
for layer in model2.layers[:5]:
    layer.trainable = False 

# Get the input
new_input = model2.input
# Find the layer to connect
hidden_layer = model2.layers[-1].output # we will connect the new layer after this
# Connect a new layer on it
new_output = Dense(no_of_features) (hidden_layer)
# Build a new model
new_model2 = Model(new_input, new_output)

#Compiling the model again
new_model2.compile(optimizer='adam', loss='mse')
new_model2.summary()

#Training the model again
fit_history2 = new_model2.fit(X_train, y_train, epochs=250, batch_size=50, 
                     validation_split=0.33, verbose=2, shuffle=False) 

#Saving the model now trainined with 31+6 columns
new_model2.save('model(8w2).h5')

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 1, 31)]           0         
                                                                 
 lstm (LSTM)                 (None, 1, 10)             1680      
                                                                 
 lstm_1 (LSTM)               (None, 7)                 504       
                                                                 
 repeat_vector (RepeatVector  (None, 2, 7)             0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 2, 7)              420       
                                                                 
 lstm_3 (LSTM)               (None, 2, 10)             720       
                                                             

In [ ]:
history = fit_history2
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=loss, name="Training Loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=val_loss, name="Validation Loss"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Training and Validation Loss"
)

# Set x-axis title
fig.update_xaxes(title_text="Epochs")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Training Loss</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Validation Loss</b>", secondary_y=True)

fig.show()

In [ ]:
#Predictions
model_pred=new_model2.predict(X_test)
print(model_pred.shape)


#Inverse Scaling of the predicted values

for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])

model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)
print(model_pred.shape)

(7, 2, 6)
(7, 2, 6)


In [ ]:
#Error Estimation

#Checking the metrics - RMSE and MAE

print('MAE_0',(MAE(newdatay[0,:], model_pred[0,:])).mean())
print('RMSE_0', (RMSE(newdatay[0,:], model_pred[0,:])).mean())

#print('RMSE', (dRMSE(newdatay, model_pred))[0,:].mean())

print('MAE_1', (MAE(newdatay[1,:], model_pred[1,:])).mean())
print('RMSE_1', (RMSE(newdatay[1,:], model_pred[1,:])).mean())

print('MAE_2', (MAE(newdatay[2,:], model_pred[2,:])).mean())

print('MAE_3', (MAE(newdatay[3,:], model_pred[3,:])).mean())

print('MAE_4', (MAE(newdatay[4,:], model_pred[4,:])).mean())

print('MAE_5', (MAE(newdatay[5,:], model_pred[5,:])).mean())

print()
print('Full MAE', (MAE(newdatay[:-2], model_pred)).mean())
print('Full RMSE', (RMSE(newdatay[:-2], model_pred)).mean())

MAE_0 0.6086954475939561
RMSE_0 5.564964546492629e-05
MAE_1 0.6962588240190478
RMSE_1 7.119092391947046e-05
MAE_2 0.8479946270450478
MAE_3 0.6547764221894198
MAE_4 0.6956229943285942
MAE_5 2.5990900543705413

Full MAE 0.9660576213984869
Full RMSE 0.00026195655383002517


In [ ]:
#Lets see the first 238 weeks data - years which we've trained our model for.
first_daily_df[:-9]


,R1,R2,R3,R4,R5,R6
0,1.485,0.050,0.530,0.120,0.915,0.160
1,1.485,0.050,0.530,0.120,0.915,0.275
2,4.005,0.050,0.535,0.360,0.915,1.330
3,2.220,0.050,0.735,1.120,4.145,1.245
4,0.620,0.050,0.535,1.230,3.285,0.840
...,...,...,...,...,...,...
233,154.070,112.170,478.165,224.790,73.910,48.255
234,164.180,111.140,492.820,220.425,74.060,51.340
235,152.535,107.145,473.240,213.730,70.950,48.685
236,160.710,114.990,492.625,231.375,74.720,47.365


In [ ]:
#Actual values for last 8 weeks - testing set
actual_df = pd.DataFrame(newdatay[:,1,:], index = list(range(238,247)))
for i in range(len(test.columns)):
    actual_df = actual_df.rename(columns={i: test.columns[i]}) 

actual_df["week"] = df.iloc[238:,0]
actual_df["week"] = pd.to_datetime(actual_df['week']).dt.date
cols = actual_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
actual_df= actual_df[cols]
actual_df


,week,R1,R2,R3,R4,R5,R6
238,2021-11-20,162.555,117.090,526.255,221.395,71.485,50.540
239,2021-11-27,164.230,113.800,527.565,226.310,71.375,50.080
240,2021-12-04,169.405,112.520,481.840,230.735,69.515,46.300
241,2021-12-11,172.085,119.235,500.885,238.895,71.415,45.505
242,2021-12-18,165.375,116.660,504.800,233.720,71.000,47.210
243,2021-12-25,113.570,76.485,337.395,161.275,55.565,35.845
244,2022-01-15,129.905,91.870,380.245,192.685,51.470,40.470
245,2022-01-22,160.965,105.090,445.630,224.780,68.220,45.205
246,2022-01-29,165.480,110.075,488.900,235.665,69.070,45.750


In [ ]:
#Predicted values for last 8 weeks
forecast1_df = pd.DataFrame(model_pred[1:,0,:], index = list(range(238,244))) 
forecast2_df = pd.DataFrame(model_pred[-5:-2,1,:], index = list(range(244,247))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 

merged["week"] = df.iloc[238:,0]
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,R1,R2,R3,R4,R5,R6
238,2021-11-20,162.211076,114.860626,524.135003,220.917153,70.886358,50.403014
239,2021-11-27,163.596814,110.881230,524.611482,225.414808,70.541678,49.836965
240,2021-12-04,169.159641,110.525716,480.006152,230.400214,68.996918,46.199702
241,2021-12-11,171.741751,117.001902,498.762474,238.417882,70.815923,45.367800
242,2021-12-18,161.334844,104.686016,491.399515,228.215331,67.288626,45.692822
243,2021-12-25,113.679185,75.309749,336.569962,161.463398,55.333493,35.876305
244,2022-01-15,170.492669,114.143307,484.293279,232.128812,70.022687,46.619034
245,2022-01-22,173.177358,120.760370,503.268805,240.392863,71.927297,45.831799
246,2022-01-29,166.470480,118.211678,507.202596,235.195132,71.512959,47.535381


In [ ]:
ffs1 = actual_df.iloc[-1:,0]+datetime.timedelta(days=7)
ffs2 = ffs1+datetime.timedelta(days=7)
ffs3 = ffs2+datetime.timedelta(days=7)
ffs4 = ffs3+datetime.timedelta(days=7)
ffs5 = ffs4+datetime.timedelta(days=7)
ffs6 = ffs5+datetime.timedelta(days=7)
ffs7 = ffs6+datetime.timedelta(days=7)
ffs8 = ffs7+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs1, ffs2, ffs3, ffs4, ffs5, ffs6, ffs7, ffs8], axis=0)
future_weeks.index = range(247,255)
future_weeks


247    2022-02-05
248    2022-02-12
249    2022-02-19
250    2022-02-26
251    2022-03-05
252    2022-03-12
253    2022-03-19
254    2022-03-26
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(247,248))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(248,255))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,R1,R2,R3,R4,R5,R6
247,2022-02-05,156.242918,112.939910,503.314415,224.213172,69.563584,48.299805
248,2022-02-12,163.643912,118.572845,528.603025,222.911786,71.994509,50.866496
249,2022-02-19,165.325016,115.356917,529.972544,227.785140,71.888289,50.405618
250,2022-02-26,170.492669,114.143307,484.293279,232.128812,70.022687,46.619034
251,2022-03-05,173.177358,120.760370,503.268805,240.392863,71.927297,45.831799
252,2022-03-12,166.470480,118.211678,507.202596,235.195132,71.512959,47.535381
253,2022-03-19,113.192398,74.947829,336.324555,160.410486,54.904759,35.612928
254,2022-03-26,130.967993,93.204513,382.465595,194.245014,51.961865,40.794326


In [ ]:
# Setting x-axis for visualization of plots
x = list(range(0, 6))
x1 = list(range(max(x)+1, max(x)+7))
x2 = list(range(max(x1)+1, max(x1)+7))
x3 = list(range(max(x2)+1, max(x2)+7))
x4 = list(range(max(x3)+1, max(x3)+7))
x5 = list(range(max(x4)+1, max(x4)+7))
x6 = list(range(max(x5)+1, max(x5)+7))
x7 = list(range(max(x6)+1, max(x6)+7))
x8 = list(range(max(x7)+1, max(x7)+7))
x9 = list(range(max(x8)+1, max(x8)+7))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

prev_val1 = first_daily_df.values[-2] # Week No 245
prev_val2 = first_daily_df.values[-1] # Week No 246

forc_val1 = model_pred[:,0,:][0] # Predicted # Week No 247
forc_val2 = model_pred[:,1,:][1] # Predicted # Week No 248
forc_val3 = model_pred[:,1,:][2] # Predicted # Week No 249
forc_val4 = model_pred[:,1,:][3] # Predicted # Week No 250

# Add traces
fig.add_trace(
    go.Scatter(x = x, y=prev_val1, name="Test - Week 245",line=dict(color="#0000ff")),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x= x1, y=prev_val2, name="Test - Week 246",line=dict(color="#0000ff")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x2, y=forc_val1, name="Forecast - Week 247",line=dict(color="darkgreen")),
    secondary_y=True,
)


fig.add_trace(
    go.Scatter(x= x3, y=forc_val2, name="Forecast - Week 248",line=dict(color="darkgreen")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x4, y=forc_val3, name="Forecast - Week 249",line=dict(color="darkgreen")),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x= x5, y=forc_val4, name="Forecast - Week 250",line=dict(color="darkgreen")),
    secondary_y=True,
)
#can be forecasted upto 36 quarters!

# Add figure title
fig.update_layout(
    title_text="Visualization of previous 2 weeks vs next 4 weeks - forecasting"
)

# Set x-axis title
fig.update_xaxes(title_text="examples")

# Set y-axes titles
fig.update_yaxes(title_text="<b>actual features</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>predictions features</b>", secondary_y=True)

fig.show()

Third Model: column "Total" (Top level in the hierarchy)

#Final Data Split (taking last feature)

In [ ]:
#last column
first_daily_df=df.iloc[:,38:]
difference=first_daily_df.diff()
first_daily_df[:-9]

,total
0,0.160
1,2.410
2,7.145
3,9.465
4,6.510
...,...
233,1091.360
234,1114.075
235,1066.285
236,1121.815


In [ ]:
n_past = 1
n_future = 8 # we will now predict for 8 future values
no_of_features = 1

#Split into the train and test data
train_dataF,test_dataF = difference[0:-9], difference[-9:] 

In [ ]:
train_dataF.columns

Index(['total'], dtype='object')

In [ ]:
#Scaling the values for faster training of the models
train = train_dataF
scalers={}
for i in train_dataF.columns:
    scaler = MinMaxScaler(feature_range=(0,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_dataF

for i in train_dataF.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
X_train, y_train = split_series(train.values,n_past, 2)

In [ ]:
X_train, y_train = split_series(train.values,n_past, 2)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],no_of_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], no_of_features))

where_are_NaNs = np.isnan(X_train)
X_train[where_are_NaNs] = 0
X_test, y_test = split_series(test.values,n_past, 2)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],no_of_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], no_of_features))
#print(y_train,X_train)

In [ ]:
print(X_train.shape)
#padding 30 zeros to match 31
z_train=np.zeros((238-2,n_past,30))
X_train = np.concatenate([X_train, z_train], -1)
print(X_train.shape)
print(y_train.shape)

(236, 1, 1)
(236, 1, 31)
(236, 2, 1)


In [ ]:
z_test=np.zeros((9-2,n_past,30))
X_test = np.concatenate([X_test, z_test], -1)
print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
newdataX,newdatay=split_series(first_daily_df.values,n_past, 2)
newdatay = newdatay[-9:, :]

(7, 1, 31)
(7, 2, 1)
(236, 1, 31)


In [ ]:
model3=load_model('model(8w2).h5') 
for layer in model3.layers[:7]:
    layer.trainable = False

# Get input
new_input = model3.input
# Find the layer to connect
hidden_layer = model3.layers[-2].output
# Connect a new layer on it
new_output = Dense(no_of_features) (hidden_layer)
# Build a new model
new_model3 = Model(new_input, new_output)
new_model3.compile(optimizer='adam', loss='mse')
new_model3.summary()

fit_history3 = new_model3.fit(X_train, y_train, epochs=300, batch_size=60, 
                     validation_split=0.33, verbose=2, shuffle=False)   #700

#Predictions on test set
model_pred=new_model3.predict(X_test)
print(model_pred.shape)


"""Inverse Scaling of the predicted values"""
for index,i in enumerate(train_dataF.columns):
    scaler = scalers['scaler_'+i]  
    model_pred[:,:,index]=scaler.inverse_transform(model_pred[:,:,index])


model_pred = [diff_inverse( model_pred[i],  newdatay[i] ) for i in range(len(model_pred))]
model_pred=np.array(model_pred)
#print(model_pred)
print(model_pred.shape)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 1, 31)]           0         
                                                                 
 lstm (LSTM)                 (None, 1, 10)             1680      
                                                                 
 lstm_1 (LSTM)               (None, 7)                 504       
                                                                 
 repeat_vector (RepeatVector  (None, 2, 7)             0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 2, 7)              420       
                                                                 
 lstm_3 (LSTM)               (None, 2, 10)             720       
                                                           

In [ ]:
history = fit_history3
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=loss, name="Training Loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=val_loss, name="Validation Loss"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Training and Validation Loss"
)

# Set x-axis title
fig.update_xaxes(title_text="Epochs")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Training Loss</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Validation Loss</b>", secondary_y=True)

fig.show()

In [ ]:
model_pred.shape

(7, 2, 1)

In [ ]:
newdatay[:-2].shape

(7, 2, 1)

In [ ]:
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)
 

dy_true = difference(newdatay[:-2])
dy_pred = difference(model_pred)

#Error Estimation
print('MAE', (MAE(newdatay[:-2], model_pred))[0,:].mean())
print('RMSE', (RMSE(newdatay[:-2], model_pred))[0,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[0,:])
print('MAE', (MAE(newdatay[:-2], model_pred))[1,:].mean())
print('RMSE', (RMSE(newdatay[:-2], model_pred))[1,:].mean())
print('dRMSE', (dRMSE(dy_true, dy_pred))[1,:])

print()
print('MAE', (MAE(newdatay[:-2], model_pred)).mean())
print('RMSE', (RMSE(newdatay[:-2], model_pred)).mean())
print('dRMSE', (dRMSE(dy_true, dy_pred)).mean())


MAE 1.4176770811267811
RMSE 0.00021491511259441216
dRMSE [0.11631994]
MAE 1.3676842777769527
RMSE 0.00019185940646392245
dRMSE [0.02658267]

MAE 1.392680679451867
RMSE 0.0002033872595291673
dRMSE 0.07145130237003106


In [ ]:
#Lets see the first 238 weeks data - years which we've trained our model for.
first_daily_df[:-9]

,total
0,0.160
1,2.410
2,7.145
3,9.465
4,6.510
...,...
233,1091.360
234,1114.075
235,1066.285
236,1121.815


In [ ]:
#Actual values for last 8 weeks - testing set
actual_df = pd.DataFrame(newdatay[:,1,:], index = list(range(238,247)))
for i in range(len(test.columns)):
    actual_df = actual_df.rename(columns={i: test.columns[i]}) 

actual_df["week"] = df.iloc[238:,0]
actual_df["week"] = pd.to_datetime(actual_df['week']).dt.date
cols = actual_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
actual_df= actual_df[cols]
actual_df

,week,total
238,2021-11-20,1150.320
239,2021-11-27,1154.960
240,2021-12-04,1111.825
241,2021-12-11,1149.715
242,2021-12-18,1141.500
243,2021-12-25,783.430
244,2022-01-15,889.465
245,2022-01-22,1052.755
246,2022-01-29,1118.480


In [ ]:
#Predicted values for last 8 weeks
forecast1_df = pd.DataFrame(model_pred[1:,0,:], index = list(range(238,244))) 
forecast2_df = pd.DataFrame(model_pred[-5:-2,1,:], index = list(range(244,247))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 

merged["week"] = df.iloc[238:,0]
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,total
238,2021-11-20,1136.226263
239,2021-11-27,1139.370443
240,2021-12-04,1098.743758
241,2021-12-11,1135.223434
242,2021-12-18,1115.123682
243,2021-12-25,772.347967
244,2022-01-15,1126.252382
245,2022-01-22,1165.347606
246,2022-01-29,1156.462649


In [ ]:
ffs1 = actual_df.iloc[-1:,0]+datetime.timedelta(days=7)
ffs2 = ffs1+datetime.timedelta(days=7)
ffs3 = ffs2+datetime.timedelta(days=7)
ffs4 = ffs3+datetime.timedelta(days=7)
ffs5 = ffs4+datetime.timedelta(days=7)
ffs6 = ffs5+datetime.timedelta(days=7)
ffs7 = ffs6+datetime.timedelta(days=7)
ffs8 = ffs7+datetime.timedelta(days=7)


future_weeks = pd.concat([ffs1, ffs2, ffs3, ffs4, ffs5, ffs6, ffs7, ffs8], axis=0)
future_weeks.index = range(247,255)
future_weeks


247    2022-02-05
248    2022-02-12
249    2022-02-19
250    2022-02-26
251    2022-03-05
252    2022-03-12
253    2022-03-19
254    2022-03-26
Name: week, dtype: object

In [ ]:
#Predicted forecasting for testing set - 8 weeeks ahead
forecast1_df = pd.DataFrame(model_pred[0:1,0,:], index = list(range(247,248))) 
forecast2_df = pd.DataFrame(model_pred[:,1,:], index = list(range(248,255))) 

for i in range(len(test.columns)):
    forecast1_df = forecast1_df.rename(columns={i: test.columns[i]})
    forecast2_df = forecast2_df.rename(columns={i: test.columns[i]}) 

merged = pd.concat([forecast1_df, forecast2_df], axis=0) 
merged["week"] = future_weeks
merged["week"] = pd.to_datetime(merged['week']).dt.date
cols = merged.columns.tolist()
cols = cols[-1:] + cols[:-1]
merged= merged[cols]
merged

,week,total
247,2022-02-05,1105.599357
248,2022-02-12,1165.861431
249,2022-02-19,1170.113584
250,2022-02-26,1126.252382
251,2022-03-05,1165.347606
252,2022-03-12,1156.462649
253,2022-03-19,791.906240
254,2022-03-26,906.014744


In [ ]:
first_daily_df.values.flatten()[:-9].shape

(238,)

In [ ]:
# Setting x-axis for visualization of plots
x = list(range(237, 237+len(model_pred.flatten())))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

prev_values = first_daily_df.values.flatten() # previous values - used of training

forc_values = model_pred.flatten()

# Add traces
fig.add_trace(
    go.Scatter(y=prev_values, name="Prev Trained Values",line=dict(color="#0000ff")),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x= x, y=forc_values, name="Forecast",line=dict(color="darkgreen")),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Visualization of forecasting"
)

# Set x-axis title
fig.update_xaxes(title_text="examples")

# Set y-axes titles
fig.update_yaxes(title_text="<b>actual features</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>predictions features</b>", secondary_y=True)

fig.show()